In [1]:
import pandas as pd
from utils import preprocess_text
from skrub import fuzzy_join, TableReport
from thefuzz import fuzz

/Users/osaid/trivago Dropbox/Omayma Said/skrub_tutorial/src/utils.py:52: SyntaxWarning: invalid escape sequence '\s'
  output_text = re.sub("\s\s+", " ", output_text)


In [2]:
## read sample data
affiliation_data_names = pd.read_csv("../data/affiliation_data_names_v1.csv")
ror_data = pd.read_csv("../data/ror_data_reference.csv")

In [3]:
## reference data
ror_data.head()

,ror_id,ror_name,ror_country_name
0,https://ror.org/04ttjf776,RMIT University,Australia
1,https://ror.org/01rxfrp27,La Trobe University,Australia
2,https://ror.org/023q4bk22,Central Queensland University,Australia
3,https://ror.org/006jxzx88,Bond University,Australia
4,https://ror.org/00wfvh315,Charles Sturt University,Australia


In [4]:
## imported data
affiliation_data_names.head()

,id,affiliation_name
0,10.17606314/h9f7p48,University of Notre Dame
1,10.52673524/j1n5b53,University of Western Brittany
2,10.65674806/h7a0b29,National Postal Museum
3,10.79962040/v3n1h38,Wageningen University & Research
4,10.7401563/n2d7d90,Shobhit Institue of Engineering and Techenology


## Process Data

Minimal processing!

In [5]:
ror_data["ror_name_processed"] = ror_data["ror_name"].apply(
    lambda x: preprocess_text(x)
)
ror_data.head()

,ror_id,ror_name,ror_country_name,ror_name_processed
0,https://ror.org/04ttjf776,RMIT University,Australia,rmit university
1,https://ror.org/01rxfrp27,La Trobe University,Australia,la trobe university
2,https://ror.org/023q4bk22,Central Queensland University,Australia,central queensland university
3,https://ror.org/006jxzx88,Bond University,Australia,bond university
4,https://ror.org/00wfvh315,Charles Sturt University,Australia,charles sturt university


In [6]:
affiliation_data_names["affiliation_name_processed"] = affiliation_data_names[
    "affiliation_name"
].apply(lambda x: preprocess_text(x))
affiliation_data_names.head()

,id,affiliation_name,affiliation_name_processed
0,10.17606314/h9f7p48,University of Notre Dame,university of notre dame
1,10.52673524/j1n5b53,University of Western Brittany,university of western brittany
2,10.65674806/h7a0b29,National Postal Museum,national postal museum
3,10.79962040/v3n1h38,Wageningen University & Research,wageningen university research
4,10.7401563/n2d7d90,Shobhit Institue of Engineering and Techenology,shobhit institue of engineering and techenology


## Fuzzy join (round 1) !

In [7]:
name_fuzzy_join_df = fuzzy_join(
    affiliation_data_names,
    ror_data,
    left_on=["affiliation_name_processed"],
    right_on=["ror_name_processed"],
    add_match_info=True,
)

In [14]:
TableReport(name_fuzzy_join_df)

Processing column  13 / 13


,,,,,,,,,,,,,


## More features

In [9]:
## add some more features to filter entries.
name_fuzzy_join_df["name_exact_match"] = name_fuzzy_join_df.apply(
    lambda x: x["affiliation_name"] == x["ror_name"], axis=1
)

name_fuzzy_join_df["name_processed_match"] = name_fuzzy_join_df.apply(
    lambda x: x["affiliation_name_processed"] == x["ror_name_processed"], axis=1
)

name_fuzzy_join_df["token_sort_ratio"] = name_fuzzy_join_df.apply(
    lambda x: fuzz.token_sort_ratio(
        x["affiliation_name_processed"], x["ror_name_processed"]
    ),
    axis=1,
)

In [15]:
TableReport(name_fuzzy_join_df)

Processing column  13 / 13


,,,,,,,,,,,,,


In [16]:
subset_examples_01 = name_fuzzy_join_df[
    (name_fuzzy_join_df["token_sort_ratio"] < 80)
    & (name_fuzzy_join_df["skrub_Joiner_distance"] < 0.3)
]

len(subset_examples_01)

11

In [17]:
TableReport(subset_examples_01[0:10])

Processing column  13 / 13


<TableReport: use .open() to display>

In [22]:
## ground truth (may include mistakes)
affiliation_data_all = pd.read_csv("../data/affiliation_data_v1.csv")

In [ ]:
## join
name_fuzzy_join_pred = name_fuzzy_join_df.merge(
    affiliation_data_all, on=["id", "affiliation_name"], how="left"
)

In [48]:
THR = 0.5
name_fuzzy_join_pred["affiliation_matched"] = name_fuzzy_join_pred.apply(
    lambda x: x["ror_id"] if x["skrub_Joiner_rescaled_distance"] < THR else None, axis=1
)

name_fuzzy_join_pred["match"] = name_fuzzy_join_pred.apply(
    lambda x: x["affiliation_id"] == x["affiliation_matched"], axis=1
)

In [46]:
# TableReport(name_fuzzy_join_pred)

In [ ]:
name_fuzzy_join_pred["match"].value_counts()

match
False    20906
True      5971
Name: count, dtype: int64

In [49]:
name_fuzzy_join_pred.groupby(
    [
        "name_processed_match",
        "match",
    ]
).size().reset_index(name="count")

,name_processed_match,match,count
0,False,False,17455
1,False,True,3983
2,True,False,345
3,True,True,5094
